In [1]:
import pandas as pd
import torch
from preprocessing import *
from LSTM import LSTM
import time
set_seed(42)

In [2]:
df = pd.read_csv(r'output_csv\ANL-Intrepid-2009-1.swf.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68936 entries, 0 to 68935
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    68936 non-null  float64
 1   submit_time               68936 non-null  float64
 2   wait_time                 68936 non-null  float64
 3   run_time                  68936 non-null  float64
 4   num_allocated_processors  68936 non-null  float64
 5   avg_cpu_time_used         68936 non-null  float64
 6   used_memory               68936 non-null  float64
 7   requested_processors      68936 non-null  float64
 8   requested_time            68936 non-null  float64
 9   requested_memory          68936 non-null  float64
 10  status                    68936 non-null  float64
 11  user_id                   68936 non-null  float64
 12  group_id                  68936 non-null  float64
 13  executable_id             68936 non-null  float64
 14  queue_

In [3]:
feature_columns = ['requested_processors', 'requested_time', 'submit_time', 'wait_time', 'user_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)
num_hidden = 64
num_layers = 3
dropout = 0.2
num_epochs = 10
lr = 0.001
batch_size = 128  
seq_len = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataloaders, scaler = prepare_data_seq(df, feature_columns, target_column, statuss=-1, seq_len=seq_len, batch_size=batch_size)

In [4]:
start = time.time()

# Initialize the model
model = LSTM(input_dim, num_hidden, num_layers, dropout).to(device)

# Train the model
model.train_model(model, dataloaders['train'], dataloaders['test'], num_epochs, lr, scaler)

end = time.time()

# Evaluate the model
rmse, mae, mse, r2 = model.evaluate_model(model, dataloaders['test'], scaler, input_dim)

print(f'\nFinal Test Metrics:')
print(f'Test RMSE: {rmse:.4f}, Test MAE: {mae:.4f}, Test MSE: {mse:.4f}, Test R2: {r2:.4f}')
print(f'Training Time: {end - start:.2f} seconds')

Epoch [1/10], Train Loss: 0.1376, Val Loss: 0.1757
Val RMSE: 7046.7864, Val MAE: 4020.3464, Val MSE: 49657199.1997, Val R2: 0.6015
Model saved at epoch 1 with validation loss: 0.1757
Epoch [2/10], Train Loss: 0.1037, Val Loss: 0.1756
Val RMSE: 7088.6480, Val MAE: 3902.8084, Val MSE: 50248930.1810, Val R2: 0.5967
Model saved at epoch 2 with validation loss: 0.1756
Epoch [3/10], Train Loss: 0.1003, Val Loss: 0.2032
Val RMSE: 7563.6213, Val MAE: 4271.9220, Val MSE: 57208366.7479, Val R2: 0.5409
Epoch [4/10], Train Loss: 0.0984, Val Loss: 0.2093
Val RMSE: 7662.6043, Val MAE: 4344.4892, Val MSE: 58715504.8216, Val R2: 0.5288
Epoch [5/10], Train Loss: 0.0969, Val Loss: 0.2143
Val RMSE: 7814.5168, Val MAE: 4447.6918, Val MSE: 61066672.3183, Val R2: 0.5099
Epoch [6/10], Train Loss: 0.0954, Val Loss: 0.2187
Val RMSE: 7884.1491, Val MAE: 4473.8066, Val MSE: 62159806.6702, Val R2: 0.5011
Epoch [7/10], Train Loss: 0.0942, Val Loss: 0.2284
Val RMSE: 8050.1381, Val MAE: 4633.5885, Val MSE: 64804723.